In [1]:
# import dependencies
# import sys
# !{sys.executable} -m pip install seaborn
import matplotlib as mplt
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import glacierml as gl
import numpy as np
import warnings
from tensorflow.python.util import deprecation
import os
import logging
import seaborn as sns
from tqdm import tqdm
from IPython.display import display, HTML
import geopy
from sklearn.cluster import KMeans

# display(HTML("<style>.container { width:85% !important; }</style>"))
tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.set_option('display.max_columns', None)

pd.set_option('mode.chained_assignment', None)

RS = range(0,25,1)

print('currently running tensorflow version: ' + tf.__version__)

2022-08-16 12:45:03.158038: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


currently running tensorflow version: 2.9.1


In [ ]:
df = gl.predictions_loader()

df = df[
    (df['dataframe'].str.contains('df7_')) 
    &
    (df['architecture'] == '10-5')
    &
    (df['learning rate'] == '0.010')
    &
    (df['epochs'] == '100')
]
x
df = gl.glathida_stats_adder(
    df,
    pth_1 = '/home/simonhans/data/prethicktor/regional_data/raw/',
    pth_2 = '/home/simonhans/data/prethicktor/RGI/rgi60-attribs/',
    pth_3 = '/home/simonhans/data/prethicktor/regional_data/training_data/',
)
df

# CLUSTER ANALYSIS

In [ ]:
#### KMeans elbow plot for RGI at region scale 


stat_list = [
    'mean',
    'median',
    'std',
    'iqr'
]
for statistic in stat_list:
    stat = statistic
    fig, ax = plt.subplots(1, 1, figsize = (20,20))
    plt.subplots_adjust(hspace=0.5)
    fig.patch.set_facecolor('w')
    dft = df[[
        'Area_RGI_' + stat,
        'Aspect_RGI_' + stat,
        'Lmax_RGI_' + stat,
        'Slope_RGI_'+ stat,
        'Zmin_RGI_' + stat,
        'Zmax_RGI_' + stat
    ]]   

    dft = dft.rename(columns = {
        'Area_RGI_' + stat:'Area',
        'Aspect_RGI_' + stat:'Aspect',
        'Lmax_RGI_' + stat:'Lmax',
        'Slope_RGI_' + stat:'Slope',
        'Zmin_RGI_' + stat:'Zmin',
        'Zmax_RGI_' + stat:'Zmax'
    })
    # ELBOW PLOT START

    x = dft.iloc[:,[0,1]]
    distortions = []
    K = range(1,10)
    for k in K:
        kmeanModel = KMeans(n_clusters = k)
        kmeanModel.fit(x)
        distortions.append(kmeanModel.inertia_)
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.suptitle(
        'The Elbow Method showing the optimal clusters for ' + stat, 
        fontsize=18, y=0.95)

# ELBOW PLOT END

    plt.show()    

In [ ]:
# plot Kmeans for RGI regions by statstical value comparing attributes 
# cannot show centroids because it won't work for reasons I do not know just yet

dft_list = [
    'Area',
    'Aspect',
    'Lmax',
    'Slope',
    'Zmin',
    'Zmax'
]
stat_list = [
    'mean',
    'median',
    'std',
    'iqr'
]

for statistic in tqdm(stat_list):
    stat = statistic



    dft = df[[
        'Area_RGI_' + stat,
        'Aspect_RGI_' + stat,
        'Lmax_RGI_' + stat,
        'Slope_RGI_'+ stat,
        'Zmin_RGI_' + stat,
        'Zmax_RGI_' + stat
    ]]   

    dft = dft.rename(columns = {
        'Area_RGI_' + stat:'Area',
        'Aspect_RGI_' + stat:'Aspect',
        'Lmax_RGI_' + stat:'Lmax',
        'Slope_RGI_' + stat:'Slope',
        'Zmin_RGI_' + stat:'Zmin',
        'Zmax_RGI_' + stat:'Zmax'
    })

    if statistic == 'mean':
        chosen_n = 7
    if statistic == 'median':
        chosen_n = 4
    if statistic == 'std':
        chosen_n = 5
    if statistic == 'iqr':
        chosen_n = 3
        
    kmeans = KMeans(n_clusters = chosen_n, random_state = 0)
    dft[stat] = kmeans.fit_predict(dft)
    residuals[stat] = dft[stat]


    for attribute in dft_list:
    

        fig, ax = plt.subplots(1, 1, figsize = (20,20))
        fig.patch.set_facecolor('w')
        plt.suptitle(
            'Cluster Analysis of RGI ' + attribute + ' ' + stat + 
            ' against RGI Attribute ' + stat, 
            fontsize=18, y=0.95
        )

        for n, att in enumerate(dft_list):
            ax = plt.subplot(3,2,n+1)

            plt.scatter(
                dft[attribute],
                dft[att],
                c = dft[stat], 
                cmap = 'viridis',
                alpha = 1, 
                marker = 'o'

            )
            ax.set_xlabel('RGI ' + stat + ' ' + attribute)
            ax.set_ylabel('RGI ' + stat + ' ' + att)
            ax.set_title('RGI ' + stat + ' ' + attribute + ' against ' + stat + ' '+ att )


            for i in df.index:
                plt.text(
                    x = dft[attribute].loc[i] + 0.02,
                    y = (
                        dft[att].loc[i]) + 0.02,
                    s = df['dataframe'].str[4:].loc[i] + '-' + str(dft[stat].loc[i])
                )
#             plt.savefig(
#                 'figs/cluster_analysis_' + 
#                 attribute + '_' + stat + '.eps'
#             )
            
            
#             plt.savefig(
#                 'figs/cluster_analysis_' + 
#                 attribute + '_' + stat + '.png'
#             )

In [ ]:
#### KMeans elbow plot for RGI statistiscs at region scale comparing statistics


att_list = [
    'Area',
    'Aspect',
    'Lmax',
    'Slope',
    'Zmin',
    'Zmax'
]
for attribute in att_list:
    dft = df[[
        attribute + '_RGI_mean',
        attribute + '_RGI_median',
        attribute + '_RGI_std',
        attribute + '_RGI_iqr',
        
    ]]

    fig, ax = plt.subplots(1, 1, figsize = (20,20))
    plt.subplots_adjust(hspace=0.5)
    fig.patch.set_facecolor('w')
    
    # ELBOW PLOT START

    x = dft.iloc[:,[0,1]]
    distortions = []
    K = range(1,10)
    for k in K:
        kmeanModel = KMeans(n_clusters = k)
        kmeanModel.fit(x)
        distortions.append(kmeanModel.inertia_)
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.suptitle(
        'The Elbow Method showing the optimal clusters for ' + attribute, 
        fontsize=18, y=0.95)

# ELBOW PLOT END

    plt.show()    

In [ ]:
# plot Kmeans for RGI regions by attribute comparing statistics 
# cannot show centroids because it won't work for reasons I do not know just yet
stat_list = [
    'mean',
    'median',
    'std',
    'iqr'
]

att_list = [
    'Area',
    'Aspect',
    'Lmax',
    'Slope',
    'Zmin',
    'Zmax'
]
for i in att_list:
    dft = df[[
        i + '_RGI_mean',
        i + '_RGI_median',
        i + '_RGI_std',
        i + '_RGI_iqr',
        
    ]]
    
    dft = dft.rename(columns = {
        i + '_RGI_mean':'mean',
        i + '_RGI_median':'median',
        i + '_RGI_std':'std',
        i + '_RGI_iqr':'iqr',
    })
    if i == 'Area':
        chosen_n =  3
    if i == 'Aspect':
        chosen_n =  4
    if i == 'Lmax':
        chosen_n =  4
    if i == 'Slope':
        chosen_n =  6
    if i == 'Zmin':
        chosen_n = 4                      
    if i == 'Zmax':
        chosen_n =  4
                     
                     
    kmeans = KMeans(n_clusters = chosen_n, random_state = 0)
    dft['cluster'] = kmeans.fit_predict(dft)
#     residuals[i] = dft[i]

    
    fig, ax = plt.subplots(1, 1, figsize = (20,20))
    fig.patch.set_facecolor('w')
    plt.suptitle(
        'Cluster Analysis of RGI ' + i + ' statistics', 
        fontsize=18, y=0.95
    )

    for n, stat in enumerate(stat_list):
        ax = plt.subplot(2,2,n+1)

        plt.scatter(
            dft['mean'],
            dft[stat],
            c = dft['cluster'], 
            cmap = 'viridis',
            alpha = 1, 
            marker = 'o'

        )
        ax.set_xlabel('RGI mean ' + ' ' + i)
        ax.set_ylabel('RGI ' + stat + ' ' + i)
        ax.set_title('RGI mean ' + i + ' against ' + stat + ' '+ i )
#         print(dft['cluster'])

        for j in (df.index):
            plt.text(
                x = dft['mean'].loc[j] + 0.02,
                y = (
                    dft[stat].loc[j]) + 0.02,
                s = df['dataframe'].str[4:].loc[j] +
                 '-' + str(dft['cluster'].loc[j])
            )
#         plt.savefig(
#             'figs/cluster_analysis_' + 
#             i + '_' + 'statistics' + '.eps'
#         )
        
        
#         plt.savefig(
#             'figs/cluster_analysis_' + 
#             i + '_' + 'statistics' + '.png'
#         )

In [ ]:
ccccombo_breaker()
# past this point is under construction. 
# here be monsters

In [ ]:
# sns density plot
sns.set(rc={"figure.figsize":(15,10)})
sns.kdeplot(x = test_labels, y = y.flatten(),fill = True)
plt.plot((0,300),(0,300),'-')

In [ ]:
# not enough data to verify physical models from just data perspective